In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
image_size = 64

train_dir = './asl/asl_alphabet_train/asl_alphabet_train/'

def get_data(dir):
    X = []
    y = []
    ctr = 0
    for cat in os.listdir(dir):
        for img_file in os.listdir(dir + cat + '/'):
            label = cat
            img = cv2.imread(dir + cat + '/' + img_file)
            img = cv2.resize(img, (image_size, image_size))
            img_arr = np.asarray(img).reshape((image_size, image_size, 3))

            X.append(img_arr)
            y.append(label)

            if ctr % 1000 == 0:
                print(str(ctr)+'/87000')
            ctr += 1
    
    return X,y

X_train, y_train = get_data(train_dir)


0/87000
1000/87000
2000/87000
3000/87000
4000/87000
5000/87000
6000/87000
7000/87000
8000/87000
9000/87000
10000/87000
11000/87000
12000/87000
13000/87000
14000/87000
15000/87000
16000/87000
17000/87000
18000/87000
19000/87000
20000/87000
21000/87000
22000/87000
23000/87000
24000/87000
25000/87000
26000/87000
27000/87000
28000/87000
29000/87000
30000/87000
31000/87000
32000/87000
33000/87000
34000/87000
35000/87000
36000/87000
37000/87000
38000/87000
39000/87000
40000/87000
41000/87000
42000/87000
43000/87000
44000/87000
45000/87000
46000/87000
47000/87000
48000/87000
49000/87000
50000/87000
51000/87000
52000/87000
53000/87000
54000/87000
55000/87000
56000/87000
57000/87000
58000/87000
59000/87000
60000/87000
61000/87000
62000/87000
63000/87000
64000/87000
65000/87000
66000/87000
67000/87000
68000/87000
69000/87000
70000/87000
71000/87000
72000/87000
73000/87000
74000/87000
75000/87000
76000/87000
77000/87000
78000/87000
79000/87000
80000/87000
81000/87000
82000/87000
83000/87000
84000

In [3]:
# Convert to np array
X_train = np.array(X_train, dtype=np.float64)
y_train = np.array(y_train)

In [4]:
X_data = X_train
y_data = y_train

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=69, stratify=y_data)

In [5]:
# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

from tensorflow.keras.utils import to_categorical
y_train_onehot = to_categorical(y_train_encoded, num_classes=29)
y_test_onehot = to_categorical(y_test_encoded, num_classes=29)


In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train_encoded.shape)
print(y_test_encoded.shape)

(65250, 64, 64, 3)
(65250,)
(21750, 64, 64, 3)
(21750,)
(65250,)
(21750,)


In [7]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(32, (5,5), input_shape=(64,64,3), data_format='channels_last'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(29, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 60, 32)        2432      
                                                                 
 activation (Activation)     (None, 60, 60, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                    

In [9]:
model.fit(X_train, y_train_onehot,
          epochs=50,
          batch_size=64,
          verbose=1,
          validation_data=(X_test, y_test_onehot),
          callbacks=[early_stop])

Epoch 1/50
1020/1020 [==============================] - 17s 12ms/step - loss: 1.2844 - accuracy: 0.6702 - val_loss: 0.2459 - val_accuracy: 0.9183
Epoch 2/50
1020/1020 [==============================] - 11s 11ms/step - loss: 0.1720 - accuracy: 0.9442 - val_loss: 0.1787 - val_accuracy: 0.9450
Epoch 3/50
1020/1020 [==============================] - 12s 11ms/step - loss: 0.1009 - accuracy: 0.9677 - val_loss: 0.0875 - val_accuracy: 0.9736
Epoch 4/50
1020/1020 [==============================] - 11s 11ms/step - loss: 0.0824 - accuracy: 0.9739 - val_loss: 0.0579 - val_accuracy: 0.9820
Epoch 5/50
1020/1020 [==============================] - 11s 11ms/step - loss: 0.0891 - accuracy: 0.9744 - val_loss: 0.1351 - val_accuracy: 0.9579
Epoch 6/50
1020/1020 [==============================] - 10s 9ms/step - loss: 0.0582 - accuracy: 0.9826 - val_loss: 0.0669 - val_accuracy: 0.9796


In [10]:
# save model
model.save('./grm')

INFO:tensorflow:Assets written to: ./grm\assets
